# MISO

In [ ]:
import urllib.request, json
import pandas as pd
from datetime import datetime, date, timedelta
import time
import sys, os
from IPython.display import display
import numpy as np
import wget
import urllib
import warnings
import requests


start_time = input('Enter the start time(YYYY-MM-DD): ')
if len(start_time) < 1:
    start_time = '2025-01-01'

end_time = input('Enter the end time(YYYY-MM-DD): ')
if len(end_time) < 1:
    end_time = '2025-01-02'

variable_name = input('Enter the Variable name: ')
if len(variable_name) < 1:
    variable_name = 'demand_forecast_hourly_regional_MISO'

info_dict_MISO = {
    'solar_forecast_hourly_MISO':{'variable_str':'_mom.xlsx','variable_date_str':'SOLAR HOURLY'},
    'wind_forecast_hourly_MISO':{'variable_str':'_mom.xlsx','variable_date_str':'WIND HOURLY'},
    'outage_forecast_daily_MISO':{'variable_str':'_mom.xlsx','variable_date_str':'OUTAGE'},
    'demand_forecast_hourly_zone_MISO':{'variable_str':'df_al.xls','variable_date_str':'n/a'},
    'demand_forecast_hourly_regional_MISO':{'variable_str':'rf_al.xls','variable_date_str':'n/a'},
    'DA_LMP_MISO':{'variable_str':'da_exante_lmp.csv','variable_date_str':'n/a'},# Miso will change it to New API
    'RT_LMP_MISO':{'variable_str':'5min_exante_lmp.xlsx','variable_date_str':'n/a'},# Miso will change it to New API
    'DA_HUB_price_MISO': {'variable_str': 'da_pr.xls','variable_date_str':'n/a'},
    'RT_HUB_price_MISO': {'variable_str': 'rt_pr.xls','variable_date_str':'n/a'},
    }

def date_range_list(start_date, end_date, days_num=1):
    # Return generator for a list datetime.date objects (inclusive) between start_date and end_date (inclusive).
    curr_date = start_date
    while curr_date <= end_date:
        yield curr_date 
        curr_date += timedelta(days=days_num)


def miso_data_downloading(start_time, end_time, variable_name, info_dict_MISO):
    start_time_list = start_time.split('-')
    end_time_list = end_time.split('-')

    d0 = date(int(start_time_list[0]), int(start_time_list[1]), int(start_time_list[2]))
    d1 = date(int(end_time_list[0]), int(end_time_list[1]), int(end_time_list[2]))
    data_array = []
    date_list = date_range_list(d0, d1)

    variable_str = info_dict_MISO[variable_name]['variable_str']
    variable_date_str = info_dict_MISO[variable_name]['variable_date_str']
    
    ###
    # variable_name=='DA_HUB_price_MISO' or variable_name=='RT_LMP_MISO'
    ###
    if variable_name=='DA_HUB_price_MISO' or variable_name=='RT_LMP_MISO':
        for day in date_list:
            date_list = str(day).split('-')
            YYYYMMDD = date_list[0]+date_list[1]+date_list[2]
            url = 'https://docs.misoenergy.org/marketreports/'+ YYYYMMDD +'_' + variable_str

            df_0 = pd.read_excel(url, index_col=None,na_values=['NA'], usecols="B")
            #print(df_0)
            header_num = 0
            header = 0
            Header_str = 'NaN'
            while Header_str != 'MISO System':
                Header_str = str(df_0.iloc[header,0])
                header = header_num
                header_num = header_num + 1
                if header > len(df_0['Unnamed: 1']):
                    break
            #print(header)
            df = pd.read_excel(io = url,index_col= None , na_values=['NA'], header=header, nrows = 24)#, usecols=["B","C","D","E","F","G","H","I","J"]
            print(df)
            if str(day) == str(start_time):
                sum_df = df
            else:
                sum_df = pd.concat([sum_df, df], ignore_index = True)

    
    
    ###
    # elif (variable_name == 'solar_forecast_hourly_MISO' or variable_name == 'wind_forecast_hourly_MISO' or variable_name == 'outage_forecast_daily_MISO')
    ###
    elif (variable_name == 'solar_forecast_hourly_MISO' or variable_name == 'wind_forecast_hourly_MISO' 
          or variable_name == 'outage_forecast_daily_MISO'):
        i = 0
        for day in date_list:
            date_str = str(day).split('-')
            YYYYMMDD = date_str[0]+date_str[1]+date_str[2]
            current_date = day + timedelta(days = 1)
            current_date_str = str(current_date).split('-')
            
            url_2 = 'https://docs.misoenergy.org/marketreports/'+ YYYYMMDD + '_'+variable_str
            print(YYYYMMDD)
            header = 0
            try:
                df = pd.read_excel(url_2, index_col=None, sheet_name=variable_date_str,na_values=['NA'], usecols="A")
                header_num = 0
                header = 0
                Header_str = 'NaN'
                while Header_str != 'DAY HE' and Header_str != 'Day HE':#or Header_str != 'Day HE'
                    Header_str = str(df.iloc[header,0])
                    header = header_num
                    header_num = header_num + 1
                    if header > len(df['Unnamed: 0']):
                        break
                df_mom = pd.read_excel(io = url_2,index_col= None , na_values=['NA'], header=header, usecols='A,B,C,D',#,D
                                     sheet_name=variable_date_str,nrows = 48,engine="openpyxl")
                time_info = 'NaN'
                time_info_1am = current_date_str[1] + '/'+ current_date_str[2] + '/' + current_date_str[0] + ' 1'
                time_info_7am = current_date_str[1] + '/'+ current_date_str[2] + '/' + current_date_str[0] + ' 7'
                
                time_idx = 0
                while time_info != time_info_1am:
                    time_info = str(''.join(list(df_mom.iloc[time_idx,0])[2:15]))
                    time_idx = time_idx + 1
                    #if time_info = 
                    if time_idx > len(df_mom.iloc[:,0]):
                        break
                iloc_start = time_idx - 1
                iloc_end = iloc_start+24
                
                
                df_mom = df_mom.iloc[iloc_start:iloc_end,:]
                data_array.append(1)
            except:
                try:
                    df = pd.read_excel(url_1, index_col=None, sheet_name=variable_date_str,na_values=['NA'], usecols="A")
                    header_num = 0
                    header = 0
                    Header_str = 'NaN'
                    while Header_str != 'DAY HE'and Header_str != 'Day HE':#or Header_str != 'Day HE'
                        Header_str = str(df.iloc[header,0])
                        header = header_num
                        header_num = header_num + 1
                        if header > len(df['Unnamed: 0']):
                            break
        
                    df_mom = pd.read_excel(io = url_1,index_col= None , na_values=['NA'], header=header, usecols='A,B,C,D',#,D
                                     sheet_name=variable_date_str,nrows = 168,engine="openpyxl")
                    time_info = 'NaN'
                    time_info_1am = current_date_str[1] + '/'+ current_date_str[2] + '/' + current_date_str[0] + ' 1'
                    time_info_7am = current_date_str[1] + '/'+ current_date_str[2] + '/' + current_date_str[0] + ' 7'
        
                    time_idx = 0
                    while time_info != time_info_1am:
                        time_info = str(''.join(list(df_mom.iloc[time_idx,0])[2:15]))
                        time_idx = time_idx + 1
                        if time_idx > len(df_mom.iloc[:,0]):
                            break
                    iloc_start = time_idx - 1
                    iloc_end = iloc_start+24
        
                    df_mom = df_mom.iloc[iloc_start:iloc_end,:]
                    data_array.append(2)
            
                except:
                    intial_date = date(int(date_str[0]), int(date_str[1]), int(date_str[2]))
                    day_i = 0
                    #day_i = day_i + 1
                    file_date = intial_date + timedelta(days = -day_i)
                    file_date_str = str(file_date).split('-')
                    YYYYMMDD_1 = file_date_str[0]+file_date_str[1]+file_date_str[2]
                    url_try = 'https://docs.misoenergy.org/marketreports/'+ YYYYMMDD_1 + variable_str
        
                    response = requests.get(url_try)
                    while response.status_code != 200:
                        file_date = intial_date + timedelta(days = -day_i)
                        file_date_str = str(file_date).split('-')
                        YYYYMMDD_1 = file_date_str[0]+file_date_str[1]+file_date_str[2]
                        url_try = 'https://docs.misoenergy.org/marketreports/'+ YYYYMMDD_1 + variable_str
                        response = requests.get(url_try)
                        day_i = day_i + 1
                        print("Found previous: ", YYYYMMDD_1)
                    #url_1_1 = 'https://docs.misoenergy.org/marketreports/'+ YYYYMMDD + '_mom.xlsx'
                    df = pd.read_excel(url_try, index_col=None, sheet_name='WIND HOURLY',na_values=['NA'], usecols="A")
                    header_num = 0
                    header = 0
                    Header_str = 'NaN'
                    while Header_str != 'DAY HE'and Header_str != 'Day HE':#or Header_str != 'Day HE'
                        Header_str = str(df.iloc[header,0])
                        header = header_num
                        header_num = header_num + 1
                        if header > len(df['Unnamed: 0']):
                            break
        
                    df_mom = pd.read_excel(io = url_try,index_col= None , na_values=['NA'], header=header, usecols='A,B,C,D',#,D
                                     sheet_name='WIND HOURLY',nrows = 168,engine="openpyxl")
                    time_info = 'NaN'
                    time_info_1am = current_date_str[1] + '/'+ current_date_str[2] + '/' + current_date_str[0] + ' 1'
                    time_info_7am = current_date_str[1] + '/'+ current_date_str[2] + '/' + current_date_str[0] + ' 7'
        
                    time_idx = 0
                    while time_info != time_info_1am:
                        time_info = str(''.join(list(df_mom.iloc[time_idx,0])[2:15]))
                        time_idx = time_idx + 1
                        if time_idx > len(df_mom.iloc[:,0]):
                            break
                    iloc_start = time_idx - 1
                    iloc_end = iloc_start+24
        
                    df_mom = df_mom.iloc[iloc_start:iloc_end,:]
                    data_array.append(2)
                if Header_str == 'Day HE':
                    df_mom = df_mom.rename(columns={'Day HE': 'DAY HE', 'North': 'North','Central': 'Central'})
            
            if str(day) == str(start_time):
                sum_df = df_mom
            else:
                sum_df = pd.concat([sum_df, df_mom], ignore_index = False)#, ignore_index = True
            url_1 = url_2
            i = i + 1
    ###
    # elif (variable_name == 'outage_forecast_daily_MISO')
    ###
    elif (variable_name == 'outage_forecast_daily_MISO'):
        i = 0
        for day in date_list:
            date_str = str(day).split('-')
            YYYYMMDD = date_str[0]+date_str[1]+date_str[2]
            current_date = day + timedelta(days = 1)
            current_date_str = str(current_date).split('-')
            
            url_2 = 'https://docs.misoenergy.org/marketreports/'+ YYYYMMDD + '_' + variable_str
            print(YYYYMMDD)
            header = 0
            try:
                #df = pd.read_excel(url_2, index_col=None, sheet_name='OUTAGE',na_values=['NA'], usecols="C")
                #header_num = 0
                header = 6
                index_day = 2
                df_outage = pd.read_excel(io = url_2,index_col= None , na_values=['NA'], header=header, usecols='A,B,C',#,D
                                     sheet_name=variable_date_str,nrows = 16,engine="openpyxl")
                
                sum_north = df_outage[df_outage["Unnamed: 0"]=="North"].sum()
                sum_central = df_outage[df_outage["Unnamed: 0"]=="Central"].sum()
                sum_south = df_outage[df_outage["Unnamed: 0"]=="South"].sum()
                sum_miso = df_outage[df_outage["Unnamed: 0"]=="MISO"].sum()
                df_sum_data_day =  pd.DataFrame({'Outage_North': [sum_north.iloc[index_day]],'Outage_Central': [sum_central.iloc[index_day]],
                               'Outage_South': [sum_south.iloc[index_day]],'Outage_MISO': [sum_miso.iloc[index_day]]})
                df_outage_sum = pd.DataFrame()
                for i_hour in range(24):
                    df_outage_sum = pd.concat([df_sum_data_day, df_outage_sum], ignore_index = False)
                
                data_array.append(1)
            except:
                try:
                    #df = pd.read_excel(url_1, index_col=None, sheet_name='SOLAR HOURLY',na_values=['NA'], usecols="A,B,C,D")
                    header_num = 0
                    header = 6
                    
                    df_outage = pd.read_excel(io = url_1,index_col= None , na_values=['NA'], header=header, usecols='A,B,C,D',#,D
                                     sheet_name=variable_date_str,nrows = 16,engine="openpyxl")
                    index_day = 3
                    sum_north = df_outage[df_outage["Unnamed: 0"]=="North"].sum()
                    sum_central = df_outage[df_outage["Unnamed: 0"]=="Central"].sum()
                    sum_south = df_outage[df_outage["Unnamed: 0"]=="South"].sum()
                    sum_miso = df_outage[df_outage["Unnamed: 0"]=="MISO"].sum()
                    df_sum_data_day =  pd.DataFrame({'Outage_North': [sum_north.iloc[index_day]],'Outage_Central': [sum_central.iloc[index_day]],
                               'Outage_South': [sum_south.iloc[index_day]],'Outage_MISO': [sum_miso.iloc[index_day]]})
                    df_outage_sum = pd.DataFrame()
                    for i_hour in range(24):
                        df_outage_sum = pd.concat([df_sum_data_day, df_outage_sum], ignore_index = False)
                    data_array.append(2)
            
                except:
                    print("Can not find: ", YYYYMMDD)
                    intial_date = date(int(date_str[0]), int(date_str[1]), int(date_str[2]))
                    day_i = 0
                    #day_i = day_i + 1
                    file_date = intial_date + timedelta(days = -day_i)
                    file_date_str = str(file_date).split('-')
                    YYYYMMDD_1 = file_date_str[0]+file_date_str[1]+file_date_str[2]
                    url_try = 'https://docs.misoenergy.org/marketreports/'+ YYYYMMDD_1+ '_'+variable_str
        
                    response = requests.get(url_try)
                    while response.status_code != 200:
                        file_date = intial_date + timedelta(days = -day_i)
                        file_date_str = str(file_date).split('-')
                        YYYYMMDD_1 = file_date_str[0]+file_date_str[1]+file_date_str[2]
                        url_try = 'https://docs.misoenergy.org/marketreports/'+ YYYYMMDD_1 + '_' + variable_str
                        response = requests.get(url_try)
                        day_i = day_i + 1
                    #print("Found previous: ", YYYYMMDD_1)
                    #url_1_1 = 'https://docs.misoenergy.org/marketreports/'+ YYYYMMDD + '_mom.xlsx'
                    #df = pd.read_excel(url_try, index_col=None, sheet_name='OUTAGE',na_values=['NA'], usecols="A,B,C,D,E,F,G,H,I")
                    header = 6
                    
                    df_outage = pd.read_excel(io = url_try,index_col= None , na_values=['NA'], header=header, usecols="A,B,C,D,E,F,G,H,I",#,D
                                     sheet_name=variable_date_str,nrows = 16,engine="openpyxl")
                    index_day = day_i+1
                    print("!!!!!",index_day)
                    
                    sum_north = df_outage[df_outage["Unnamed: 0"]=="North"].sum()
                    sum_central = df_outage[df_outage["Unnamed: 0"]=="Central"].sum()
                    sum_south = df_outage[df_outage["Unnamed: 0"]=="South"].sum()
                    sum_miso = df_outage[df_outage["Unnamed: 0"]=="MISO"].sum()
                    df_sum_data_day =  pd.DataFrame({'Outage_North': [sum_north.iloc[index_day]],'Outage_Central': [sum_central.iloc[index_day]],
                               'Outage_South': [sum_south.iloc[index_day]],'Outage_MISO': [sum_miso.iloc[index_day]]})
                    df_outage_sum = pd.DataFrame()
                    for i_hour in range(24):
                        df_outage_sum = pd.concat([df_sum_data_day, df_outage_sum], ignore_index = False)
                    print(df_outage_sum)
                data_array.append(2)
                
            
            if str(day) == str(start_time):
                sum_df = df_outage_sum
            else:
                sum_df = pd.concat([sum_df, df_outage_sum], ignore_index = False)#, ignore_index = True
            url_1 = url_2
            i = i + 1
    ###
    # elif (variable_name == 'demand_forecast_hourly_zone_MISO')
    ###
    elif variable_name == 'demand_forecast_hourly_zone_MISO':
        for day in date_list:
            date_list = str(day).split('-')
            YYYYMMDD = date_list[0]+date_list[1]+date_list[2]
            url = 'https://docs.misoenergy.org/marketreports/'+ YYYYMMDD + '_' + variable_str
        
            df = pd.read_excel(io = url,index_col= None , na_values=['NA'], 
                               header=None, skiprows = [i for i in (range(32))], nrows = 24).drop([3,5,7,9,11,13,15], axis=1)#, usecols=["C","E","G","I","K","M","O"]
            df.rename(columns={0: 'Date', 1: 'Hour', 2: 'Zone1', 4: 'Zone2_7', 6: 'Zone3_5', 8: 'Zone4', 10: 'Zone6', 12: 'Zone8_9', 14: 'MISO'}, inplace=True)#print(day)
            #print(df)
            if str(day) == str(start_time):
                sum_df = df
            else:
                sum_df = pd.concat([sum_df, df], ignore_index = True)

    ###
    # elif (variable_name == 'demand_forecast_hourly_zone_MISO')
    ###
    elif (variable_name == 'demand_forecast_hourly_regional_MISO'):
        for day in date_list:
            date_list = str(day).split('-')
            YYYYMMDD = date_list[0]+date_list[1]+date_list[2]
            url = 'https://docs.misoenergy.org/marketreports/'+ YYYYMMDD + '_' + variable_str
    
            df = pd.read_excel(io = url,index_col= None , na_values=['NA'],# North - D; Central - F; MISO - J
                               header=None, skiprows = [i for i in (range(34))], nrows = 24).drop([0, 4,6,8,10], axis=1)
            df.rename(columns={1: 'Date', 2: 'Hour', 3: 'North', 5: 'Central', 7: 'South', 9: 'MISO'}, inplace=True)
            #print(day)
            if str(day) == str(start_time):
                sum_df = df
            else:
                sum_df = pd.concat([sum_df, df], ignore_index = True)
        
    return sum_df


sum_df = miso_data_downloading(start_time, end_time, variable_name, info_dict_MISO)
print(sum_df)

# PJM

In [ ]:
import urllib.request, json
import pandas as pd
from datetime import datetime, timedelta,date
import time
import random

start_time = input('Enter the start time(YYYY-MM-DD): ')
if len(start_time) < 1:
    start_time = '2024-01-01'

end_time = input('Enter the end time(YYYY-MM-DD): ')
if len(end_time) < 1:
    end_time = '2025-10-01'

variable_name = input('Enter the Variable name: ')
if len(variable_name) < 1:
    variable_name = 'demand_forecast_hourly_PJM'

info_dict_PJM = {'solar_forecast_5min_PJM':{'variable_str':'five_min_solar_power_forecast','variable_date_str':'datetime_beginning_utc'},
    'solar_forecast_hourly_PJM':{'variable_str':'hourly_solar_power_forecast','variable_date_str':'datetime_beginning_utc'},
    'wind_forecast_5min_PJM':{'variable_str':'five_min_wind_power_forecast','variable_date_str':'datetime_beginning_utc'},
    'wind_forecast_hourly_PJM':{'variable_str':'hourly_wind_power_forecast','variable_date_str':'datetime_beginning_utc'},
    'outage_forecast_daily_PJM':{'variable_str':'frcstd_gen_outages','variable_date_str':'forecast_date'},
    'demand_forecast_hourly_PJM':{'variable_str':'load_frcstd_hist','variable_date_str':'forecast_hour_beginning_utc'},
    'DA_LMP_PJM':{'variable_str':'da_hrl_lmps','variable_date_str':'datetime_beginning_utc'},
    'RT_LMP_PJM':{'variable_str':'rt_fivemin_mnt_lmps','variable_date_str':'datetime_beginning_utc'},
    }
def date_range_list(start_date, end_date, days_num=1):
    # Return generator for a list datetime.date objects (inclusive) between start_date and end_date (inclusive).
    curr_date = start_date
    while curr_date <= end_date:
        yield curr_date 
        curr_date += timedelta(days=days_num)
    if curr_date != end_date:
        yield end_date

def pjm_data_downloading(start_time, end_time, variable_name, info_dict_PJM):
    variable_str = info_dict_PJM[variable_name]['variable_str']
    variable_date_str = info_dict_PJM[variable_name]['variable_date_str']
    PJM_no_hist_variable_list = ['solar_forecast_5min_PJM','solar_forecast_hourly_PJM','wind_forecast_5min_PJM','wind_forecast_hourly_PJM']
    
    hdr ={
            # Request headers
            'Cache-Control': 'no-cache',
            'Ocp-Apim-Subscription-Key': '336423db3cd243a3841d772330f22cc7',
            }
    urls = []
    df_results = pd.DataFrame()
    
    ####################################
    if variable_name in PJM_no_hist_variable_list:
        print("Note: PJM does not store renewable forecast data beyond 30 days, the function will turn to download last 30 days' data")
        current_date = datetime.now()# Get the current date
        formatted_current_date = datetime.now().strftime("%Y-%m-%d")
        formatted_30_days_ago_date = (datetime.now()-timedelta(days=30)).strftime("%Y-%m-%d")
        if (variable_name == 'solar_forecast_5min_PJM' or variable_name == 'wind_forecast_5min_PJM'):
        ## 5 min level data beyond the capacity of single PJM API call, turn it to mutiple API call
            try:
                for iter_days in range(9):
                    start_time_iter = (current_date-timedelta(days=iter_days*3+3)).strftime("%Y-%m-%d")
                    end_time_iter = (current_date-timedelta(days=iter_days*3)).strftime("%Y-%m-%d")
                    url = ("https://api.pjm.com/api/v1/" + variable_str +
                           "?rowCount=50000&order=Asc&startRow=1&" + variable_date_str + 
                           "=" + start_time_iter + "%2005:00%20to%20" + end_time_iter + "%2004:00")
                    urls.append(url)
            
            except Exception as e:
                print(e)
        else:
            try:
                start_time_iter = current_date
                url = ("https://api.pjm.com/api/v1/" + variable_str +
                       "?rowCount=50000&order=Asc&startRow=1&" + variable_date_str + 
                       "=" + formatted_30_days_ago_date + "%2005:00%20to%20" + formatted_current_date + "%2004:00")
                urls.append(url)
            
            except Exception as e:
                print(e)
    ####################################       
    elif variable_name == 'outage_forecast_daily_PJM':
        try:
            url = ("https://api.pjm.com/api/v1/" + variable_str +
                   "?rowCount=50000&order=Asc&startRow=1&" + variable_date_str + 
                   "=" + start_time)
            
            urls.append(url)
        except Exception as e:
            print(e)
    
    ####################################
    elif variable_name == 'DA_LMP_PJM'or variable_name == 'RT_LMP_PJM':  
        try:
            start_time_list = start_time.split('-')
            end_time_list = end_time.split('-')
            d0 = date(int(start_time_list[0]), int(start_time_list[1]), int(start_time_list[2]))
            d1 = date(int(end_time_list[0]), int(end_time_list[1]), int(end_time_list[2]))
            
            if variable_name == 'DA_LMP_PJM':#increase download threadhold
                days_num = 300
            else:
                days_num = 150
            
            date_var_list = date_range_list(d0, d1, days_num= days_num)
            urls = []
            iter_num = 0
            for i_date in date_var_list:
                #print(str(i_date),iter_num)
                if iter_num == 0:
                    start_d = str(i_date)
                elif iter_num != 0:
                    end_d = str(i_date)
                    if variable_name == 'DA_LMP_PJM':
                        url = ("https://api.pjm.com/api/v1/" + variable_str +
                                   "?rowCount=50000&order=Asc&startRow=1&" + variable_date_str + 
                                   "=" + start_d + "%2005:00%20to%20" + end_d + "%2004:00"+"&pnode_id=1")
                    elif variable_name == 'RT_LMP_PJM':
                        url = ("https://api.pjm.com/api/v1/" + variable_str +
                                   "?rowCount=50000&order=Asc&startRow=1&" + variable_date_str + 
                                   "=" + start_d + "%2005:00%20to%20" + end_d + "%2004:55"+"&pnode_id=1")
                    urls.append(url)
                    start_d = str(i_date)
                iter_num = iter_num + 1
            #urls.append(url)
        except Exception as e:
            print(e)
    ####################################
    else:  
        try:
            start_time_list = start_time.split('-')
            end_time_list = end_time.split('-')
            d0 = date(int(start_time_list[0]), int(start_time_list[1]), int(start_time_list[2]))
            d1 = date(int(end_time_list[0]), int(end_time_list[1]), int(end_time_list[2]))
            if variable_name == 'DA_LMP_PJM':
                days_num 
            date_var_list = date_range_list(d0, d1, days_num=7)
            urls = []
            iter_num = 0
            for i_date in date_var_list:
                #print(str(i_date),iter_num)
                if iter_num == 0:
                    start_d = str(i_date)
                elif iter_num != 0:
                    end_d = str(i_date)
                    url = ("https://api.pjm.com/api/v1/" + variable_str +
                               "?rowCount=50000&order=Asc&startRow=1&" + variable_date_str + 
                               "=" + start_d + "%2005:00%20to%20" + end_d + "%2004:00")
                    print(url)
                    urls.append(url)
                    start_d = str(i_date)
                iter_num = iter_num + 1
            
            #urls.append(url)
        except Exception as e:
            print(e)
    ####################################
    for i_url, url in enumerate(urls):
        #time.sleep(random.randint(1, 5))
        req = urllib.request.Request(url, headers=hdr)
        req.get_method = lambda: 'GET'
        response = urllib.request.urlopen(req)
        print(response.getcode())
        results = response.read()
        json_data = json.loads(results.decode('utf-8'))
        print(url)
        df_results_1 = pd.DataFrame(json_data['items'])
        print(df_results_1)
        if i_url % 4 == 0 and i_url!=0:#the whole loop used to reset the PJM api call
            time.sleep(random.randint(60, 62)) #take a sleep to make it like human bechavior
            url_random = "https://api.pjm.com/api/v1/load_frcstd_hist?rowCount=50000&order=Asc&startRow=1"
            req = urllib.request.Request(url_random, headers=hdr)#take a random api call to reset the count
            req.get_method = lambda: 'GET'
            response = urllib.request.urlopen(req)
            print('Reset the api count')
        df_results = pd.concat([df_results,df_results_1], ignore_index=True)
    #print(df_results)
    
    return df_results
df_results = pjm_data_downloading(start_time, end_time, variable_name, info_dict_PJM)
print(df_results)

# National Weather Service

In [ ]:
import numpy as np
import http.client
from urllib.parse import urlparse
variable_name = input('Enter the Variable Name (string): ')
if len(variable_name) < 1:
    variable_name = 'Temp'

year_str = input('Enter the year (string): ')
if len(year_str) < 1:
    year_str = '2024'

info_dict_NWS = {'Temp':{'variable_str':'temp','file_str': "YEUZ98_KWBN_"},
    'DewPoint':{'variable_str':'td','file_str': "YFUZ98_KWBN_"},
    'WindDirection':{'variable_str':'wdir','file_str': "YBUZ98_KWBN_"},
    'WindSpeed':{'variable_str':'wspd','file_str': "YCUZ98_KWBN_"},
    'SkyCover':{'variable_str':'sky','file_str': "YAUZ98_KWBN_"},
    'RelativeHumidity':{'variable_str':'rhm','file_str': "YRUZ98_KWBN_"},
    'ApparentTemp':{'variable_str':'apt','file_str': "YTUZ98_KWBN_"},
    }

#variable_str = info_dict_NWS[variable_name]['variable_str']
#path_str = "https://noaa-ndfd-pds.s3.amazonaws.com/wmo/"+ variable_str + "/"
#file_str = info_dict_NWS[variable_name]['file_str']

def check_link_exists(url):
    try:
        parsed_url = urlparse(url)
        conn = http.client.HTTPSConnection(parsed_url.netloc)
        conn.request("HEAD", parsed_url.path)
        response = conn.getresponse()
        return response.status == 200
    except Exception as e:
        print(f"Error: {e}")
        return False

def check_file_name_previous(path_str, file_str,time_str, year_str, month_str, day_str, hour_int):#Used if the current hour's file is not exist
    file_hour_index = 0
    mins = 0
    result = False
    hour_delta = 1
    url_index = (path_str + time_str + file_str + year_str 
                     + month_str + day_str + str(hour_int-hour_delta))
    while result != True:
        if mins>= 60:
            mins = 0
            hour_delta = hour_delta + 1
            hour_idx = hour_int-hour_delta
            url_index = (path_str + time_str + file_str + year_str
                     + month_str + day_str + str(hour_idx))
            #print("error")
            #break
        if mins < 10:
            mins_str = "0"+ str(mins)
        else:
            mins_str = str(mins)
        url = url_index + mins_str
        result = check_link_exists(url)
        mins = mins + 1
    file_exist_url = url_index + mins_str#str(mins-1)
    return file_exist_url, hour_delta

def check_file_name(path_str, file_str, time_str, year_str, month_str, day_str, hour_int):
    file_hour_index = 0
    mins = 0
    result = False
    url_index = (path_str + time_str + file_str + year_str 
                     + month_str + day_str + str(hour_int))
    
    while result != True:
        if mins>= 60:
            print("error:current hour's weather not availible")
            url_previous, file_hour_index = check_file_name_previous(path_str, file_str, time_str, year_str, month_str, day_str, hour_int)
            #url_previous 
            file_exist_url = url_previous #url_index + str(mins-1)
            return file_exist_url, file_hour_index
            break
        if mins < 10:
            mins_str = "0"+ str(mins)
        else:
            mins_str = str(mins)
        url = url_index + mins_str
        result = check_link_exists(url)
        mins = mins + 1
    file_exist_url = url_index + mins_str#str(mins-1)
    return file_exist_url, file_hour_index
#print(check_file_name(url_index))


def download_links_list_generation(info_dict,variable_name_str,hour_int,year_str):
    
    '''
    info_dict: dict for the variable infomation list for 
    variable_name_str: which variable are downloading ('Temp','DewPoint','WindDirection','WindSpeed','SkyCover','RelativeHumidity','ApparentTemp')
    hour_int: which hour as the reference for the forcast scan
    year_str: which year you want downloading
    '''
    variable_str = info_dict[variable_name_str]['variable_str']
    path_str = "https://noaa-ndfd-pds.s3.amazonaws.com/wmo/"+ variable_str + "/"
    file_str = info_dict[variable_name_str]['file_str']
    url_list = []
    days_num_month = np.array([31,28,31,30,31,30,31,31,30,31,30,31])
    
    for i_month in range(12):
        for i_day in range(days_num_month[i_month]):
            if i_month <9:
                month_str = '0'+str(i_month+1)
            else:
                month_str = str(i_month+1)
            if i_day <9:
                day_str = '0'+str(i_day+1)
            else:
                day_str = str(i_day+1)
            time_str = year_str+'/'+ month_str +'/'+day_str+'/'
            #url_index = ("https://noaa-ndfd-pds.s3.amazonaws.com/wmo/temp/" + time_str + "YEUZ98_KWBN_"+ year_str 
            #             + month_str + day_str + "20")
            result, file_hour_index = check_file_name(path_str, file_str,time_str, year_str, month_str, day_str, hour_int)#url_index
            print(result,file_hour_index)
            url_list.append(result)
            #print(check_file_name(url_index))
    return url_list

hour_int = 20
url_list = download_links_list_generation(info_dict_NWS,variable_name,hour_int,year_str)
print(url_list)